In [3]:
import sys
sys.path.insert(0, "..")
from utils import linalg
from importlib import reload
import distributed
from distributed import kernel
reload(distributed.sharded_matrix)
from distributed.sharded_matrix import ShardedMatrix, ShardedSymmetricMatrix
from distributed import matmul
import numpy as np
import pywren
from utils import misc
import math
import concurrent.futures as fs
import time
import os
from numba import jit
import scipy.linalg
from sklearn import metrics
from scipy.linalg import LinAlgError

from distributed import bcd
reload(bcd)

<module 'distributed.bcd' from '../distributed/bcd.py'>

In [4]:
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [5]:
X_EXISTS = True
K_EXISTS = True
NORMS_EXIST = True
GAMMA = 1e-1
NUM_TRAIN = 1281167

In [6]:
pwex = pywren.default_executor(job_max_runtime=600)

In [7]:
%time X_block_sharded = ShardedMatrix("X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017",\
                                shard_size_0=4096,\
                                bucket="vaishaalpywrenlinalg",\
                                n_jobs=32)

CPU times: user 64 ms, sys: 0 ns, total: 64 ms
Wall time: 180 ms


In [8]:
key = "rbf(XXT(X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_blocks_no_normalize_04.22.2017, {0})".format(GAMMA)

if (not K_EXISTS):
   K = kernel.compute_kernel_pywren(pwex, key, X_block_sharded, 4, gamma=GAMMA, num_jobs=4500, num_features=X_block_sharded.shape[1])
else: 
    K = ShardedSymmetricMatrix(key, bucket="vaishaalpywrenlinalg")

In [9]:
K.get_block(312,312)

array([[ 1.        ,  0.95819718,  0.93563199, ...,  0.94763786,
         0.93435502,  0.91795731],
       [ 0.95819718,  1.        ,  0.95140797, ...,  0.971856  ,
         0.94155627,  0.93967021],
       [ 0.93563199,  0.95140797,  1.        , ...,  0.96874422,
         0.97331804,  0.9642204 ],
       ..., 
       [ 0.94763786,  0.971856  ,  0.96874422, ...,  1.        ,
         0.94539356,  0.95176649],
       [ 0.93435502,  0.94155633,  0.97331804, ...,  0.94539356,
         1.        ,  0.96965617],
       [ 0.91795731,  0.93967021,  0.9642204 , ...,  0.95176649,
         0.96965617,  1.        ]], dtype=float32)

In [12]:
idxs = np.load("/home/ubuntu/imagenet-data/scramble_idxs.npy")
labelsTrain = np.load("/home/ubuntu/imagenet-data/imagenet_train_labels.npy")[idxs]

In [13]:
labelsTrain

array([ 955.,  978.,   98., ...,  752.,   91.,  239.])

In [14]:
y_train = np.eye(1000)[labelsTrain.astype('int')]

In [15]:
K.get_block(0,0)

array([[ 1.        ,  0.94733649,  0.9838956 , ...,  0.91740602,
         0.96881169,  0.95540237],
       [ 0.94733649,  1.        ,  0.9513042 , ...,  0.92391437,
         0.9574669 ,  0.98219329],
       [ 0.9838956 ,  0.9513042 ,  1.        , ...,  0.92284334,
         0.97720796,  0.96359736],
       ..., 
       [ 0.91740602,  0.92391437,  0.92284334, ...,  0.99999994,
         0.91570693,  0.91001534],
       [ 0.96881169,  0.9574669 ,  0.97720796, ...,  0.91570693,
         1.        ,  0.96754104],
       [ 0.95540237,  0.98219329,  0.96359736, ...,  0.91001534,
         0.96754104,  1.        ]], dtype=float32)

In [73]:
reload(bcd)

<module 'distributed.bcd' from '../distributed/bcd.py'>

In [ ]:
sys.stdout = terminal_stdout
reload(bcd)
X_1_epoch = bcd.block_kernel_solve(K, y_train, epochs=2, lambdav=1e-3, blocks_per_iter=10, x_start=X_1_epoch[0], y_hat=X_1_epoch[1], sb=240)

../distributed/bcd.py:50: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (x_start == None):
Process Process-294:
KeyboardInterrupt
Process Process-308:
Process Process-307:
Process Process-300:
Process Process-304:
Process Process-298:
Process Process-289:


KeyboardInterrupt: 

Process Process-299:
Process Process-305:
Process Process-301:
Process Process-297:
Process Process-296:
Process Process-290:
Process Process-306:
Process Process-302:
Process Process-303:
Process Process-295:
Process Process-293:
Process Process-291:
Process Process-288:
Process Process-292:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process

In [1]:
model = np.load("model_iter_29_train_acc_0.48995017823593645_next_block_acc_0.212676426755501_norm_1790426.3223825234.npy")

NameError: name 'np' is not defined

In [21]:
metrics.accuracy_score(np.argmax(X_1_epoch[1], axis=1), labelsTrain)

0.38887436220258559

In [23]:
K_test_sharded = ShardedMatrix("rbf(XYT(X_block_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017, X_test_scrambled_3_3_pool_12_12_patch_stride_1_1_block_no_normalize_04.22.2017), 0.1)",\
                               bucket="vaishaalpywrenlinalg")


In [64]:
%time futures = bcd.fast_kernel_column_block_async(K_test_sharded, col_blocks=K_test_sharded._block_idxs(1), mmap_loc="/dev/shm/K_test_block", dtype="float64")
%time fs.wait(futures)

DoneAndNotDoneFutures(done={<Future at 0x7fa480981048 state=finished returned tuple>, <Future at 0x7fa722b0c0b8 state=finished returned tuple>, <Future at 0x7fa480981160 state=finished returned tuple>, <Future at 0x7fa480981208 state=finished returned tuple>, <Future at 0x7fa2180b1a20 state=finished returned tuple>, <Future at 0x7fa4809812b0 state=finished returned tuple>, <Future at 0x7fa2180b1ac8 state=finished returned tuple>, <Future at 0x7fa480981358 state=finished returned tuple>, <Future at 0x7fa2180b1b70 state=finished returned tuple>, <Future at 0x7fa480981400 state=finished returned tuple>, <Future at 0x7fa2180b1c18 state=finished returned tuple>, <Future at 0x7fa4809814a8 state=finished returned tuple>, <Future at 0x7fa2180b1cc0 state=finished returned tuple>, <Future at 0x7fa480981550 state=finished returned tuple>, <Future at 0x7fa2180b1d68 state=finished returned tuple>, <Future at 0x7fa4809815f8 state=finished returned tuple>, <Future at 0x7fa2180b1e10 state=finished ret

In [26]:
%time X_test = np.load("/home/ubuntu/imagenet-data/X_test_lift_3_3_pool_12_12_patch_stride_1.npy")
np.random.seed(0)
idxs_test = np.random.choice(X_test.shape[0], X_test.shape[0], replace=False)
labels_test = np.load("/home/ubuntu/imagenet-data/imagenet_test_labels.npy")
labels_test = labels_test[idxs_test]

In [67]:
model = X_1_epoch[0].astype('float32')
%time y_test_pred = K_test.T.dot(model)

In [75]:
K_test_sharde

In [65]:
%time K_test = bcd.load_mmap(*futures[0].result())

In [63]:
np.save("/home/ubuntu/model_iter_20_train_acc_0.3533278643611645_next_block_acc_0.1871337890625_norm_710921.825824806.npy", model)

In [66]:
K_test.shape

(1281167, 50000)

In [68]:
metrics.accuracy_score(np.argmax(y_test_pred, axis=1), labels_test)

0.17976

In [22]:
model = np.load("/dev/shm/model_iter_70_train_acc_0.809784360664925_next_block_acc_0.137939453125_norm_725.7616952103908.npy")

In [164]:
svd_stuff = np.linalg.svd(X_1_epoch)

In [172]:
svd_stuff[1][0]/svd_stuff[1][-1]

10493230837679216.0

In [177]:
np.diag(X_1_epoch)

memmap([ 0.38845128,  0.89646912,  0.44712415, ...,  0.26498708,
        0.56446284,  0.66869539])

In [179]:
K_block = K.get_block(0,0)

In [187]:
np.diag(K_block)

array([ 0.99999881,  0.99999988,  0.99999988, ...,  1.        ,
        0.99999952,  1.        ], dtype=float32)

In [183]:
svd_stuff[1]

array([  2.17837646e+03,   3.11883759e+02,   1.85900482e+02, ...,
         2.88411011e-05,   2.49138957e-05,   1.49347890e-17], dtype=float32)

In [42]:
x_start, y_hat =  X_1_epoch

In [145]:
%time a = bcd.fast_kernel_column_block_async(K, [0,1,2])
%time fs.wait(a)

[0, 1, 2]
CPU times: user 276 ms, sys: 1.78 s, total: 2.06 s
Wall time: 2.04 s
CPU times: user 912 ms, sys: 84 ms, total: 996 ms
Wall time: 1min 45s


DoneAndNotDoneFutures(done={<Future at 0x7f2d00221898 state=finished returned tuple>, <Future at 0x7f2ce04cf898 state=finished returned tuple>, <Future at 0x7f2ce04cf0b8 state=finished returned tuple>, <Future at 0x7f2de057b8d0 state=finished returned tuple>, <Future at 0x7f2ce04cf9e8 state=finished returned tuple>, <Future at 0x7f1f5df06a20 state=finished returned tuple>, <Future at 0x7f2ce04cfa58 state=finished returned tuple>, <Future at 0x7f2d00221a58 state=finished returned tuple>, <Future at 0x7f1f5d3a72b0 state=finished returned tuple>, <Future at 0x7f2cc2186b70 state=finished returned tuple>, <Future at 0x7f2dc0295470 state=finished returned tuple>, <Future at 0x7f2de057bc88 state=finished returned tuple>, <Future at 0x7f2dc02954a8 state=finished returned tuple>, <Future at 0x7f2ccba2e4e0 state=finished returned tuple>, <Future at 0x7f2cc993ad30 state=finished returned tuple>, <Future at 0x7f38c1699d68 state=finished returned tuple>, <Future at 0x7f2ce04cf588 state=finished ret

In [146]:
k = bcd.load_mmap(*a[0].result())

In [152]:
bcd.block_kernel_solve(K, 

memmap([[ 0.99999881,  0.40342984,  0.36615327, ...,  0.30506128,
         0.67470342,  0.40624243],
       [ 0.40342984,  0.99999988,  0.64286745, ...,  0.64839852,
         0.69248831,  0.80197585],
       [ 0.36615327,  0.64286745,  0.99999988, ...,  0.68896604,
         0.72726387,  0.8055684 ],
       ..., 
       [ 0.33696416,  0.68765193,  0.73388755, ...,  0.874237  ,
         0.59649545,  0.85790408],
       [ 0.35511884,  0.82535362,  0.73779732, ...,  0.74089462,
         0.64481544,  0.86099726],
       [ 0.28665397,  0.70327604,  0.67107129, ...,  0.90301561,
         0.5732618 ,  0.85399121]])

In [38]:
k

NameError: name 'k' is not defined